# Декомпозиция Клементса

In [1]:
"""
Clements unitary decomposition
"""

from typing import NamedTuple, List, Tuple
import torch
import numpy as np
from torch import Tensor


def mzi_matrix(
    dim: int,
    target: Tuple[int, int],
    theta: float,
    phi: float,
    dtype=torch.complex128,
    device='cpu'
) -> Tensor:
    """Matrix form of a Mach-Zehnder interferometer using PyTorch."""
    m, n = target
    if n - m != 1:
        raise ValueError("m and n must be consecutive integers.")

    u_mzi = torch.eye(dim, dtype=dtype, device=device)

    theta_t = torch.tensor(theta, dtype=torch.float32, device=device, requires_grad=False)
    phi_t = torch.tensor(phi, dtype=torch.float32, device=device, requires_grad=False)

    global_phase = torch.exp(1j * theta_t)
    cos_t = torch.cos(theta_t)
    sin_t = torch.sin(theta_t)
    exp_phi = torch.exp(1j * phi_t)

    u_mzi[m, m] = global_phase * exp_phi * cos_t
    u_mzi[m, n] = global_phase * 1j * sin_t
    u_mzi[n, m] = global_phase * exp_phi * 1j * sin_t
    u_mzi[n, n] = global_phase * cos_t

    return u_mzi


class MachZehnder(NamedTuple):
    theta: float
    phi: float
    target: Tuple[int, int]


class Decomposition(NamedTuple):
    D: Tensor
    circuit: List[MachZehnder]


def _is_unitary(U: Tensor, atol=1e-6) -> bool:
    I = torch.eye(U.shape[0], dtype=U.dtype, device=U.device)
    return torch.allclose(U @ U.conj().T, I, atol=atol)

def clements_decomposition(U: Tensor) -> Decomposition:
    if not isinstance(U, torch.Tensor):
        raise TypeError("U must be a torch.Tensor.")
    if not _is_unitary(U):
        raise ValueError("U must be a unitary matrix.")

    dim = U.shape[0]
    dtype = U.dtype
    device = U.device

    right_sequence: List[MachZehnder] = []
    left_sequence: List[MachZehnder] = []
    U = U.clone()

    for k in range(1, dim):
        if k % 2 == 1:
            for i, j in zip(range(dim - 1, dim - k - 1, -1), range(k - 1, -1, -1)):
                if torch.abs(U[i, j]) < 1e-12:
                    continue
                if torch.abs(U[i, j + 1]) < 1e-12:
                    phi = 0.0
                else:
                    phi = (torch.angle(U[i, j] / U[i, j + 1]) - torch.pi / 2) % (2 * torch.pi)
                theta = torch.arctan2(torch.abs(U[i, j]), torch.abs(U[i, j + 1]))
                right_sequence.append(MachZehnder(theta, phi, (j, j + 1)))
                mzi = mzi_matrix(dim, (j, j + 1), theta, phi, dtype=dtype, device=device)
                U = U @ mzi.conj().T
        else:
            for i, j in zip(range(dim - k, dim), range(0, k)):
                if torch.abs(U[i, j]) < 1e-12:
                    continue
                if torch.abs(U[i - 1, j]) < 1e-12:
                    phi = 0.0
                else:
                    phi = (torch.angle(U[i, j] / U[i - 1, j]) + torch.pi / 2) % (2 * torch.pi)
                theta = torch.arctan2(torch.abs(U[i, j]), torch.abs(U[i - 1, j]))
                left_sequence.insert(0, MachZehnder(theta, phi, (i - 1, i)))
                mzi = mzi_matrix(dim, (i - 1, i), theta, phi, dtype=dtype, device=device)
                U = mzi @ U

    D = torch.diag(U).clone()
    new_left_sequence: List[MachZehnder] = []

    for theta, phi, (m, n) in left_sequence:
        new_phi = (torch.pi + torch.angle(D[m]) - torch.angle(D[n])) % (2 * torch.pi)
        new_left_sequence.insert(0, MachZehnder(theta, new_phi, (m, n)))
        new_beta = torch.angle(D[n]) - 2 * theta
        new_alpha = torch.angle(D[n]) - torch.pi - phi - 2 * theta
        D[m] = torch.exp(1j * new_alpha)
        D[n] = torch.exp(1j * new_beta)

    return Decomposition(D=D, circuit=new_left_sequence + right_sequence[::-1])

def circuit_reconstruction(decomposition: Decomposition) -> Tensor:
    if not isinstance(decomposition, Decomposition):
        raise TypeError("decomposition must be a Decomposition object.")

    D = decomposition.D
    dim = D.shape[0]
    dtype = D.dtype
    device = D.device

    reconstructed_matrix = torch.diag(D)

    for theta, phi, (m, n) in decomposition.circuit:
        mzi = mzi_matrix(dim, (m, n), theta, phi, dtype=dtype, device=device)
        reconstructed_matrix = reconstructed_matrix @ mzi

    return reconstructed_matrix

# Разложение Река

In [2]:
def t_matrix(
    dim: int,
    target: Tuple[int, int],
    theta: Tensor,   # теперь принимаем Tensor (для градиентов)
    phi: Tensor,     # тоже Tensor
    dtype=torch.complex128,
    device='cpu'
) -> Tensor:
    """
    Matrix form of a 2-mode unitary transformation T_{m,n}(theta, phi)
    used in Reck decomposition.

    The 2x2 block is:
        [[cos(theta),        -exp(-i*phi) * sin(theta)],
         [exp(i*phi) * sin(theta),  cos(theta)]]

    Parameters:
        dim: размер полной матрицы
        target: кортеж (m, n) — индексы мод
        theta: угол вращения (скалярный тензор, dtype=float)
        phi: фазовый угол (скалярный тензор, dtype=float)
        dtype: тип комплексных чисел (например, torch.complex128)
        device: устройство

    Returns:
        Полная dim x dim унитарная матрица.
    """
    m, n = target

    # Создаём единичную матрицу нужного размера
    T = torch.eye(dim, dtype=dtype, device=device)

    # Вычисляем элементы 2x2 блока с использованием torch (дифференцируемо!)
    cos_t = torch.cos(theta)
    sin_t = torch.sin(theta)
    exp_iphi = torch.exp(1j * phi)

    # Заполняем 2x2 подматрицу
    T[m, m] = cos_t
    T[m, n] = -torch.conj(exp_iphi) * sin_t  # = -exp(-i*phi) * sin(theta)
    T[n, m] = exp_iphi * sin_t               # =  exp(i*phi) * sin(theta)
    T[n, n] = cos_t

    return T

def reck_decomposition(U: Tensor):
    """
    Reck decomposition of a unitary matrix U into a sequence of
    2-mode transformations T_{m,n}(theta, phi) and final phases.
    
    Returns:
        thetas: Tensor of rotation angles (real, shape [K])
        phis:   Tensor of phase angles (real, shape [K])
        targets: Tensor of mode pairs (int64, shape [K, 2])
        phases: Diagonal phase shifts (real, shape [N])
    """
    N = U.shape[0]
    device = U.device
    dtype_real = torch.float64
    dtype_int = torch.int64

    # Инициализация списков (лучше, чем cat в цикле)
    thetas_list = []
    phis_list = []
    targets_list = []

    U_work = U.clone()

    for n in range(N - 1, 0, -1):          # столбцы: справа налево
        for m in range(n - 1, -1, -1):     # строки: снизу вверх в столбце n
            a = U_work[n, n]
            b = U_work[m, n]

            r = torch.sqrt(torch.abs(a)**2 + torch.abs(b)**2 + 1e-15)

            if r < 1e-12:
                theta = torch.tensor(0.0, dtype=dtype_real, device=device)
                phi = torch.tensor(0.0, dtype=dtype_real, device=device)
            else:
                cos_theta = torch.abs(a) / r
                cos_theta = torch.clamp(cos_theta, -1.0, 1.0)
                theta = torch.acos(cos_theta)
                phi = torch.angle(a) - torch.angle(b)  # ВАЖНО: правильный знак!

            thetas_list.append(theta)
            phis_list.append(phi)
            targets_list.append(torch.tensor([m, n], dtype=dtype_int, device=device))

            # Строим T_{m,n}
            T_mn = t_matrix(N, target=(m,n), theta=theta, phi=phi)

            U_work = T_mn @ U_work

    # Собираем тензоры
    thetas = torch.stack(thetas_list) if thetas_list else torch.empty(0, dtype=dtype_real, device=device)
    phis = torch.stack(phis_list) if phis_list else torch.empty(0, dtype=dtype_real, device=device)
    targets = torch.stack(targets_list) if targets_list else torch.empty((0, 2), dtype=dtype_int, device=device)

    phases = torch.angle(torch.diag(U_work))  # вещественные фазы

    return thetas, phis, targets, phases

def get_matrix(
    dim: int, 
    phases: Tensor,
    phis: Tensor,
    thetas: Tensor,
    targets: Tensor,
    dtype=torch.complex128,
    device='cpu'
) -> Tensor:
    U = torch.diag(torch.exp(1j * phases.to(dtype))).to(device)
    
    # Применяем в ОБРАТНОМ порядке! (Reck применял T @ U, значит U = T1† T2† ... D)
    for i in range(len(thetas) - 1, -1, -1):
        theta = thetas[i]
        phi = phis[i]
        m, n = targets[i].tolist()
        T_mn = t_matrix(dim, target=(m,n), theta=theta, phi=phi)

        U = T_mn.adjoint() @ U

    return U

# Проверка (Клементс)

In [3]:
torch.manual_seed(42)

def fidellity(U_exp: Tensor, U: Tensor):
    return torch.abs(torch.einsum('ii', U_exp.adjoint() @ U)).item()/U.shape[0]

def haar_measure(N):
    """Generate a Haar-random matrix using the QR decomposition."""
    A, B = torch.randn(size=(N, N), dtype=torch.float64), torch.randn(size=(N, N), dtype=torch.float64)
    Z = A + 1j * B
    Q, R = torch.linalg.qr(Z)
    Lambda = torch.diag(torch.tensor([R[i, i] / torch.abs(R[i, i]) for i in range(N)]))

    return Q @ Lambda

N = torch.randint(low=2, high=20, size = ())
U = haar_measure(N)

print("Initial Unitary:\n", U.numpy().round(4))

# Apply Clements decomposition
decomposition = clements_decomposition(U)

# Print the decomposition parameters
print("\nCircuit:\n")
num_layer = 1
temp = 0
print(f'Layer {num_layer}\n')
for theta, phi, target in decomposition.circuit:
    if temp > target[0]:
        num_layer += 1
        print(f'\nLayer {num_layer}\n')
    print(f"theta: {theta:.3f}, phi: {phi:.2f}, target: {target}")
    temp = target[0]

# Reconstruct the unitary from the decomposition
reconstructed_unitary = circuit_reconstruction(decomposition)

# Print the reconstructed unitary and assert initial matrix
print("\nReconstructed Unitary:\n", reconstructed_unitary.numpy().round(4))
print("\nFidellity:\n", fidellity(reconstructed_unitary, U))
assert torch.allclose(U, reconstructed_unitary, atol=1e-06), "Reconstructed unitary does not match original."

Initial Unitary:
 [[-0.2006+0.0945j  0.5269-0.099j  -0.2327+0.2757j  0.008 +0.1147j
  -0.0506+0.092j  -0.1041-0.2304j -0.3972+0.4805j -0.0954-0.2177j]
 [-0.1295-0.1587j  0.1815+0.1986j -0.0996-0.1983j -0.3697+0.4263j
  -0.1729-0.019j  -0.4346+0.3077j  0.2055+0.1765j  0.1376+0.3344j]
 [-0.3724-0.5003j -0.1125-0.2725j -0.0402+0.2994j -0.3849-0.0267j
   0.393 +0.2562j  0.1423+0.1155j -0.0635-0.1336j -0.0753+0.0523j]
 [-0.1528+0.2489j -0.0655+0.3764j  0.1486+0.1059j -0.2019-0.1234j
  -0.3107+0.3058j  0.3503+0.3919j  0.0303+0.1637j -0.4256-0.0662j]
 [-0.0238-0.1727j -0.3071+0.078j   0.1853-0.2987j  0.0207+0.0415j
   0.0188+0.4263j  0.1027-0.1532j  0.0292+0.4863j  0.4608-0.2788j]
 [ 0.1624+0.2351j  0.2311-0.0187j  0.1415-0.227j  -0.1423-0.3131j
   0.5125-0.1013j -0.0614+0.5134j -0.2805+0.1268j  0.1911-0.0567j]
 [-0.0476+0.3983j -0.4043-0.1464j -0.6899-0.1172j -0.0803-0.118j
   0.0564+0.1781j -0.0128-0.0217j -0.0923+0.1006j  0.0269+0.2987j]
 [ 0.3913+0.092j  -0.2228-0.1365j  0.0504-0.0933j -0

/var/folders/86/5xcctjn15052xcf5hwdbv_6h0000gp/T/ipykernel_45199/511947677.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  theta_t = torch.tensor(theta, dtype=torch.float32, device=device, requires_grad=False)
/var/folders/86/5xcctjn15052xcf5hwdbv_6h0000gp/T/ipykernel_45199/511947677.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  phi_t = torch.tensor(phi, dtype=torch.float32, device=device, requires_grad=False)


## Проверка (Рек)

In [4]:
N = torch.randint(low=2, high=20, size = ())
U = haar_measure(N)

print("Initial Unitary:\n", U.numpy().round(4))
thetas, phis, targets, phases = reck_decomposition(U)

# Print the decomposition parameters
print("\nCircuit:\n")

for i in range(len(thetas) - 1, -1, -1):
    print(f'\nLayer {len(thetas) - i}\n')
    theta = thetas[i]
    phi = phis[i]
    m, n = targets[i].tolist()
    print(f"theta: {theta:.3f}, phi: {phi:.2f}, target: {(m, n)}")

reconstructed_unitary = get_matrix(N, phases, phis, thetas, targets)

# Print the reconstructed unitary and assert initial matrix
print("\nReconstructed Unitary:\n", reconstructed_unitary)
print("\nFidellity:\n", fidellity(reconstructed_unitary, U))
assert torch.allclose(U, reconstructed_unitary, atol=1e-06), "Reconstructed unitary does not match original."

Initial Unitary:
 [[ 0.0073+0.2203j -0.1411-0.0886j  0.061 +0.4463j  0.3052+0.0673j
   0.2207-0.1099j  0.2861+0.3648j  0.5891-0.0168j]
 [-0.1708-0.1095j  0.0113+0.1097j -0.5437+0.1249j  0.4255-0.0532j
   0.4129-0.1876j  0.1136-0.4419j -0.1646-0.1037j]
 [ 0.5238+0.3781j  0.1524+0.0299j -0.2022-0.1863j  0.0586-0.1991j
   0.1443+0.2397j -0.4261-0.1808j  0.3528-0.1514j]
 [-0.0537+0.4461j -0.3094+0.0577j -0.138 -0.122j   0.3473+0.1255j
  -0.2015-0.09j   -0.1966+0.4026j -0.4233-0.3164j]
 [-0.3494-0.2535j -0.0515+0.5575j  0.1013+0.1824j  0.1135-0.1974j
   0.0013+0.5223j -0.2371+0.1213j  0.1165-0.218j ]
 [-0.0466+0.1526j -0.1023+0.5499j -0.3911-0.0521j -0.5374+0.0769j
  -0.1116-0.3511j  0.1269+0.0644j  0.2241-0.0712j]
 [ 0.1117+0.2583j -0.456 -0.0519j  0.0994+0.409j  -0.3873+0.2057j
   0.3112+0.3183j  0.0217-0.2652j -0.2629-0.0508j]]

Circuit:


Layer 1

theta: 0.647, phi: 4.80, target: (0, 1)

Layer 2

theta: 1.021, phi: -4.38, target: (0, 2)

Layer 3

theta: 0.993, phi: -5.39, target: (1, 2)